# Consultas anidadas (Subqueries)

Las consultas anidadas son una herramienta flexible y en el cual se pueden utilizar en 4 clausulas principales de SQL (SELECT, INSERT, UPDATE, DELETE). Una consulta anidada corresponde a una subconsulta contenida en otra sentencia SQL, donde a esta última la llamaremos la sentencia contenedora. **Las subconsultas están siempre contenidas dentro de paréntesis** y usualmente son ejecutadas previamente a la sentencia contenedora. Estas pueden retornarnos un conjunto los cuales pueden tener:

- Una fila y una columna
- Múltiples filas y una columna
- Múltiples filas y múltiples columnas

Según el conjunto retornado por la subconsulta, podernos determinar cómo pueden ser utilizadas y que operadores de la sentencia contenedora, pueden interactuar con los datos retornados. Cuando la sentencia contenedora es ejecutada en su totalidad, todos los valores retornados por una subconsulta **son descartados**

La sintaxis de una consulta anidada corresponde a la siguiente:

```SQL
SELECT columna1, columna2, ..., columnaN
FROM tabla_A
WHERE columna1 = (
    SELECT columna1
    FROM tabla_B
);
```

En este caso tenemos la consulta contenedora como: 

```SQL
SELECT columna1, columna2, ..., columnaN
FROM tabla_A
WHERE columna1 = 
```

y la subconsulta como:
```SQL
 (
    SELECT columna1
    FROM tabla_B
);
```

In [1]:
import sqlite3 
import pandas as pd

conn = sqlite3.connect("db//Chinook_Sqlite.sqlite") 
cursor = conn.cursor()

In [2]:
#Imprimimos el catalogo de tablas solo para hacer Check de que todo cargo correctamente
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall()) 

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


Realicemos una primera consulta anidada, supongamos que obtener el Id, El primer nombre y el apellido del cliente con mayor valor de Id (El ultimo cliente de la tabla Customer). Si lo hiciésemos en dos partes, tendríamos dos consultas del estilo:

In [3]:
sqlQuery = """
       SELECT MAX(customerId)
       FROM customer
"""
pd.read_sql(sqlQuery,conn)

,MAX(customerId)
0,59


In [4]:
sqlQuery = """
    SELECT customerId, firstName, lastName
    FROM customer
    WHERE customerId = 59
"""
pd.read_sql(sqlQuery,conn)

,CustomerId,FirstName,LastName
0,59,Puja,Srivastava


Por otra parte, con las consultas anidadas, nosotros podemos generar la cláusula contenedora, y la cláusula anidada la cual retorne el valor de inmediato:

In [5]:
sqlQuery = """
    SELECT customerId,
       firstName,
       lastName
    FROM customer
    WHERE customerId = (
       SELECT MAX(customerId)
       FROM customer
       );
"""
pd.read_sql(sqlQuery,conn)

,CustomerId,FirstName,LastName
0,59,Puja,Srivastava


Alternativamente y ya que estamos trabajando con SQLite, podemos utilizar funciones de agregación y obtener dicho valor. Pero **recordar** que la siguiente consulta solo funciona en SQLite (por la propiedad de permitir columnas agrupadas y no agrupadas a la vez), en otros motores de base de datos, **La siguiente consulta se debería caer**. Por lo mismo una alternativa a esta consulta la cual si funciona, corresponde a la consulta anterior.

In [6]:
sqlQuery = """
    SELECT MAX(customerId),
       firstName,
       lastName
    FROM customer
"""
pd.read_sql(sqlQuery,conn)

,MAX(customerId),FirstName,LastName
0,59,Puja,Srivastava


Veamos otro ejemplo utilizando la tabla Track, en este caso, seleccionemos todas las canciones pertenecientes al disco 'Ride The Lightning':

In [7]:
sqlQuery = """
    SELECT trackid,
       name,
       albumid
    FROM track
    WHERE albumid = (
       SELECT albumid
       FROM album
       WHERE title = 'Ride The Lightning'
    );
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,1874,Fight Fire With Fire,154
1,1875,Ride The Lightning,154
2,1876,For Whom The Bell Tolls,154
3,1877,Fade To Black,154
4,1878,Trapped Under Ice,154
5,1879,Escape,154
6,1880,Creeping Death,154
7,1881,The Call Of Ktulu,154


Alternativamente, nosotros podríamos escribir la consulta anidada, utilizando joins y luego haciendo la cláusula `WHERE` para obtener el nombre de album.

**Nota:** No siempre podremos traspasar una consulta anidada a una consulta utilizando joins (de una manera tan sencilla como este ejemplo). Todo va a depender de la complejidad de las subconsultas las cuales nosotros vayamos generando.

In [8]:
sqlQuery = """
    SELECT trackid,
       name,
       albumid
    FROM track
    NATURAL JOIN Album a
    WHERE a.title = 'Ride The Lightning'
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,1874,Fight Fire With Fire,154
1,1875,Ride The Lightning,154
2,1876,For Whom The Bell Tolls,154
3,1877,Fade To Black,154
4,1878,Trapped Under Ice,154
5,1879,Escape,154
6,1880,Creeping Death,154
7,1881,The Call Of Ktulu,154


## Tipos de consultas anidadas

Existen dos tipos de consultas anidadas: **las correlacionadas** y las **no correlacionadas**. Hasta ahora, todas las consultas que hemos visto corresponden a consultas **no correlacionadas**, las cuales se caracterizan por no referenciar nada de la consulta contenedora. Por otra parte, las consultas anidadas **correlacionadas** hacen referencia a la consulta contenedora, pero esto lo vamos a ver más adelante.

Otro tipo de consultas anidadas, pueden ser las **consultas escalares**, el resultado de la subconsulta es simplemente una fila y una columna, otro ejemplo de una consulta escalar:

In [9]:
sqlQuery = """
    SELECT trackid,
       name,
       albumid
    FROM track
    WHERE albumid != (
       SELECT albumid
       FROM album
       WHERE title = 'Ride The Lightning'
    );
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,1,For Those About To Rock (We Salute You),1
1,2,Balls to the Wall,2
2,3,Fast As a Shark,3
3,4,Restless and Wild,3
4,5,Princess of the Dawn,3
...,...,...,...
3490,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343
3491,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344
3492,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345
3493,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346


Las subconsultas *no escalares*, también conocidas como subconsultas que tienen múltiples filas, requieren ciertos operadores para poder hacer la comparación. En la mayoría de los motores de base de datos, nosotros no podríamos hacer una consulta del siguiente tipo: 

```SQL
SELECT trackid,
   name,
   albumid
FROM track
WHERE albumid = (
   SELECT albumid
   FROM album
   WHERE title <> 'Ride The Lightning'
);
```
La subconsulta retorna un subconjunto de Identificadores, y técnicamente nosotros no podríamos igualar un identificador a un conjunto de identificadores, por lo que tendríamos que utilizar la sentencia **IN** (y su negación **NOT IN**).

**Nota:** Si nosotros corremos la consulta anterior, SQLite no generara ningún problema, esto es debido al punto 10 de los [Quirks de SQLITE](https://www.sqlite.org/quirks.html)

In [10]:
#Utilizando la sintaxis correcta
sqlQuery = """
    SELECT trackid,
       name,
       albumid
    FROM track
    WHERE albumid IN (
       SELECT albumid
       FROM album
       WHERE title <> 'Ride The Lightning'
    );
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,1,For Those About To Rock (We Salute You),1
1,6,Put The Finger On You,1
2,7,Let's Get It Up,1
3,8,Inject The Venom,1
4,9,Snowballed,1
...,...,...,...
3490,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343
3491,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344
3492,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345
3493,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346


## subconsultas correlacionadas

Las subconsultas correlacionadas, tienen explícitamente una condición entre la subconsulta y la consulta contenedora. Cabe detallar que, si nosotros utilizamos subconsultas correlacionadas, la subconsulta se va a ejecutar por cada fila candidata que pueda cumplir las condiciones. 

Veamos un ejemplo obteniendo todas las canciones las cuales estén exactamente en 5 playlists:


In [11]:
sqlQuery = """
    SELECT t.TrackId,
       name,
       albumid
    FROM Track t
    WHERE (
       SELECT COUNT(*)
       FROM PlayListTrack plt
       WHERE plt.TrackId = t.TrackId
    ) = 5 ;
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,3403,Intoitus: Adorate Deum,272
1,3404,"Miserere mei, Deus",273
2,3408,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",277
3,3409,"Suite for Solo Cello No. 1 in G Major, BWV 100...",278
4,3410,"The Messiah: Behold, I Tell You a Mystery... T...",279
5,3411,Solomon HWV 67: The Arrival of the Queen of Sheba,280
6,3415,Symphony No.5 in C Minor: I. Allegro con brio,284
7,3416,Ave Maria,285
8,3417,"Nabucco: Chorus, ""Va, Pensiero, Sull'ali Dorate""",286
9,3418,Die Walküre: The Ride of the Valkyries,287


In [12]:
sqlQuery = """
    SELECT t.TrackId,
       name,
       albumid,
       (
       SELECT COUNT(*)
       FROM PlayListTrack plt
       WHERE plt.TrackId = t.TrackId
       ) as counts
    FROM Track t
    ;
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,counts
0,1,For Those About To Rock (We Salute You),1,3
1,2,Balls to the Wall,2,3
2,3,Fast As a Shark,3,4
3,4,Restless and Wild,3,4
4,5,Princess of the Dawn,3,4
...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,5
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,4
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,4
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,4


El operador exists nos permite obtener la comprobación de que una relación existe, independiente de la cantidad que existan al momento de hacer la subconsulta. Esto ocurre ya sea si seleccionamos una, muchas o ninguna de las columnas presentes en una tabla.

In [13]:
sqlQuery = """
    SELECT t.TrackId,
       t.name,
       t.albumid
    FROM Track t
    INNER JOIN Genre g
    USING(GenreId)
    WHERE EXISTS
    (
        SELECT 1 FROM PlaylistTrack plt
        WHERE plt.TrackId = t.Trackid
            AND g.Name = 'Blues'
    )
    ;
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,194,First Time I Met The Blues,20
1,195,Let Me Love You Baby,20
2,196,Stone Crazy,20
3,197,Pretty Baby,20
4,198,When My Left Eye Jumps,20
...,...,...,...
76,2586,Twice As Hard,210
77,2587,Lickin',210
78,2588,Soul Singing,210
79,2589,Hard To Handle,210


## Inserción, Modificación y eliminación con subconsultas

Unas de las características principales de las subconsultas es la versatilidad al poder retornar elementos que nosotros queramos en la subconsulta, y luego manipular dichos resultados en la consulta contenedora. En este caso, nosotros podemos utilizar las sentencias `INSERT`, `UPDATE` y `DELETE` en conjunto con las subconsultas para adecuar según nuestras necesidades las distintas operaciones que vamos a aplicar. En primer lugar, vamos a hacer un respaldo de la tabla Tracks, ya que vamos a realizar operaciones modificando dicha tabla. Para esto, vamos a extraer el SQL utilizado para crear dicha tabla y generar un duplicado de esta (pero con otro nombre)


**Nota:** en otros motores de base de datos, existen operaciones similares para obtener el SQL que crea una tabla, por ejemplo mysql implementa `SHOW CREATE TABLE tbl_name`

In [14]:
# Seleccionamos todo desde el catalogo, pero especificamente que el nombre de la tabla sea Track y extraemos el SQL que la crea
sql_Track = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table' AND name = 'Track';",conn).sql[0]
print(sql_Track)

CREATE TABLE [Track]
(
    [TrackId] INTEGER  NOT NULL,
    [Name] NVARCHAR(200)  NOT NULL,
    [AlbumId] INTEGER,
    [MediaTypeId] INTEGER  NOT NULL,
    [GenreId] INTEGER,
    [Composer] NVARCHAR(220),
    [Milliseconds] INTEGER  NOT NULL,
    [Bytes] INTEGER,
    [UnitPrice] NUMERIC(10,2)  NOT NULL,
    CONSTRAINT [PK_Track] PRIMARY KEY  ([TrackId]),
    FOREIGN KEY ([AlbumId]) REFERENCES [Album] ([AlbumId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([GenreId]) REFERENCES [Genre] ([GenreId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([MediaTypeId]) REFERENCES [MediaType] ([MediaTypeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


In [15]:
#Reemplazamos el nombre de dicha tabla, con el fin de que no existan problemas de tablas duplicadas
sql_Track = sql_Track.replace("[Track]","[Track_BKP]")

Recordar que, para escribir cambios a disco, nosotros tenemos que utilizar el método de la conexión `commit`. Todos los cambios sin salvar se escriben en disco y quedan persistentemente para consultas futuras

In [16]:
#Creamos la tabla Track_BKP
cursor.execute(sql_Track)
conn.commit()

In [17]:
#Revisamos si esta la nueva tabla que acabamos de crear 

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall()) 

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',), ('Track_BKP',)]


Supongamos que queremos respaldar todos los datos de la tabla Track, hacia Track_BKP, esto lo podemos realizar con la siguiente sentencia

In [18]:
sqlQuery = """
INSERT INTO Track_BKP 
    SELECT * FROM Track
"""

cursor.execute(sqlQuery)

Luego podemos consultar si está poblada o no la tabla Track_BKP, haciendo un `SELECT`

In [19]:
sqlQuery = """
    SELECT * FROM Track_BKP
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


Ahora supongamos que en verdad no queremos guardar todos los registros, sino más bien los que vamos a ir actualizando, con el fin de mantener su estado inicial. Podríamos eliminar todos los registros de nuestra tabla utilizando la sentencia DELETE. **PERO**, como todavía nosotros no hacemos un commit, podemos hacer una operación alternativa. La operación `rollback()` nos permitirá devolver a su estado original todos los cambios hechos y que no hayan pasado por un commit. En este caso vamos a llamar a `conn.rollback()` para dejar la tabla Track_BKP vacía.

In [20]:
conn.rollback()

In [21]:
#Examinamos la tabla
sqlQuery = """
    SELECT * FROM Track_BKP
"""
pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice


Con todo esto, ahora hagamos el ejercicio de hacer un descuento para las Canciones de los grupos AC/DC y Metallica. En primer lugar necesitamos obtener todos los IDs de las canciones de ambos grupos:

In [22]:
sqlQuery = """
    SELECT t.TrackId, a.Name from Track T
    INNER JOIN Album al 
    ON t.AlbumId = al.AlbumId
    INNER JOIN Artist a
    ON al.ArtistId = a.ArtistId
    WHERE a.Name IN ('AC/DC','Metallica')
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name
0,1,AC/DC
1,6,AC/DC
2,7,AC/DC
3,8,AC/DC
4,9,AC/DC
...,...,...
125,1897,Metallica
126,1898,Metallica
127,1899,Metallica
128,1900,Metallica


Ahora utilizando los IDs obtenidos anteriormente, podemos insertar en la tabla de Backup solo las canciones que vamos a modificar:

In [23]:
sqlQuery = """
INSERT INTO Track_BKP 
    SELECT * FROM Track
    WHERE Track.trackId IN 
    (
        SELECT t.TrackId from Track T
        INNER JOIN Album al 
        ON t.AlbumId = al.AlbumId
        INNER JOIN Artist a
        ON al.ArtistId = a.ArtistId
        WHERE a.Name IN ('AC/DC','Metallica')
    )
    
"""

cursor.execute(sqlQuery)

In [24]:
#Revisamos que solo esten canciones de AC/DC
sqlQuery = """
    SELECT * FROM Track_BKP
"""
pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99
...,...,...,...,...,...,...,...,...,...
125,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.99
126,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.99
127,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.99
128,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.99


In [25]:
#Escribimos en la base de datos los cambios
conn.commit()

Ahora, con los datos respaldados, podremos hacer el proceso de modificar la tabla, primero definiremos la subquery que nos devolverá todos los identificadores para modificar

In [26]:
subSqlQuery = """
        SELECT t.TrackId from Track T
        INNER JOIN Album al 
        ON t.AlbumId = al.AlbumId
        INNER JOIN Artist a
        ON al.ArtistId = a.ArtistId
        WHERE a.Name IN ('AC/DC','Metallica')
"""

y utilizando `format` podemos incluir fácilmente el string de la subconsulta, en la consulta contenida. En este caso, vamos a bajar un $50\%$ el valor unitario de las canciones de AC/DC y Metallica

In [27]:
sqlQuery = f"""
UPDATE Track 
    SET UnitPrice = UnitPrice * 0.5
    WHERE Track.trackId IN 
    ({subSqlQuery})
    
"""

cursor.execute(sqlQuery)

Ahora revisamos todas las canciones que tengan un valor menor al mínimo antiguo (que era 0.99) y podemos revisar que son todas canciones de AC/DC o Metallica

In [28]:

sqlQuery = """
    SELECT * FROM Track Where UnitPrice < 0.99    
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.495
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.495
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.495
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.495
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.495
...,...,...,...,...,...,...,...,...,...
125,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.495
126,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.495
127,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.495
128,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.495


En un cambio de espíritu, al gerente de la empresa se dio cuenta que la verdad no le gusta tanto AC/DC para poder hacer el descuento a dicha banda. Por lo mismo nos ha pedido revertir los cambios, pero solo para dicha banda. Entonces nosotros podemos redefinir la subconsulta y aplicar tanto las cláusulas de UPDATE y DELETE donde ocurran las condiciones que nosotros queramos (Que sean de la banda AC/DC).

In [29]:
subSqlQuery = """
        SELECT t.TrackId from Track t
        INNER JOIN Album al 
        ON t.AlbumId = al.AlbumId
        INNER JOIN Artist a
        ON al.ArtistId = a.ArtistId
        WHERE a.Name = 'AC/DC'
"""

#Devolvemos al precio Original, como posible mejora de la bitacora, podemos extraer dicho valor de la tabla de Backup
sqlQuery1 = f"""
UPDATE Track 
    SET UnitPrice = UnitPrice * 2
    WHERE trackId IN 
    ({subSqlQuery})
"""

#Eliminamos las canciones que ya no necesitamos Backup
sqlQuery2 = f"""
DELETE FROM Track_BKP 
    WHERE trackId IN 
    ({subSqlQuery})
"""
cursor.execute(sqlQuery1)
cursor.execute(sqlQuery2)

Ahora vemos que las canciones que quedan, son solo de metallica tanto en precio, como en la tabla de backup

In [30]:
sqlQuery = """
    SELECT * FROM Track Where UnitPrice < 0.99    
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,408,Free Speech For The Dumb,35,1,3,Molaney/Morris/Roberts/Wainwright,155428,5076048,0.495
1,409,It's Electric,35,1,3,Harris/Tatler,213995,6978601,0.495
2,410,Sabbra Cadabra,35,1,3,Black Sabbath,380342,12418147,0.495
3,411,Turn The Page,35,1,3,Seger,366524,11946327,0.495
4,412,Die Die My Darling,35,1,3,Danzig,149315,4867667,0.495
...,...,...,...,...,...,...,...,...,...
107,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.495
108,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.495
109,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.495
110,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.495


In [31]:
sqlQuery = """
    SELECT * FROM Track_BKP
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,408,Free Speech For The Dumb,35,1,3,Molaney/Morris/Roberts/Wainwright,155428,5076048,0.99
1,409,It's Electric,35,1,3,Harris/Tatler,213995,6978601,0.99
2,410,Sabbra Cadabra,35,1,3,Black Sabbath,380342,12418147,0.99
3,411,Turn The Page,35,1,3,Seger,366524,11946327,0.99
4,412,Die Die My Darling,35,1,3,Danzig,149315,4867667,0.99
...,...,...,...,...,...,...,...,...,...
107,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.99
108,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.99
109,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.99
110,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.99


In [32]:
#Guardamos los cambios
conn.commit()

In [33]:
sqlQuery = """
    SELECT * FROM Track_BKP
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,408,Free Speech For The Dumb,35,1,3,Molaney/Morris/Roberts/Wainwright,155428,5076048,0.99
1,409,It's Electric,35,1,3,Harris/Tatler,213995,6978601,0.99
2,410,Sabbra Cadabra,35,1,3,Black Sabbath,380342,12418147,0.99
3,411,Turn The Page,35,1,3,Seger,366524,11946327,0.99
4,412,Die Die My Darling,35,1,3,Danzig,149315,4867667,0.99
...,...,...,...,...,...,...,...,...,...
107,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.99
108,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.99
109,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.99
110,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.99
